In [ ]:
# default_exp analyze

# analyze

> Tools for computing movement/pose similarity time series, clustering for single- and multi-dancer videos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from openpifpaf.datasets.constants import COCO_KEYPOINTS, COCO_PERSON_SKELETON

# May not need all of these here...
#import io
import numpy as np
#import PIL
from PIL import Image
#import pickle
#import matplotlib.pyplot as plt
import math
import cv2
import os

import warnings
warnings.filterwarnings(
  action='ignore', module='matplotlib.figure', category=UserWarning,
  message=('This figure includes Axes that are not compatible with tight_layout, '
           'so results might be incorrect.'))

from scipy.interpolate import interp1d

def smooth_series(x, window_len=11, window='flat'):
    """ Smooth a time series via a sliding window average
        From https://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
        Generally used as a helper function to process_time_series() below.
    """
    if x.ndim != 1:
        raise(ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise(ValueError, "Input vector needs to be bigger than window size.")

    if window_len<3:
        print("WARNING: window length too small for smoothing, returning input data")
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise(ValueError, "Window is one of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')

    # Move the window 1/2 width back to avoid lag
    if window_len % 2 == 0:
        return y[int(window_len/2)-1:-(int(window_len/2))]
    else:
        return y[int(window_len/2):-(int(window_len/2))]


def corr_time_series_matrix(pose_data, method='distance'):
    """ Generate a full time-series pose similarity heatmap for all available
        poses and frames from the video. This code can use either pose
        characterization approach; in practice, the distance matrix-based analyses
        take longer to calculate but are more accurate.
    """
    pose_correlations = []
    for i, pi in enumerate(pose_data):
        print("Comparing frame",i,"to the rest")
        corr_row = []
        if method == 'distance':
            mi = get_pose_matrix(pi)
        else: # method == 'laplacian'
            mi = get_laplacian_matrix(pi)
        for j, pj in enumerate(pose_data):
            if j < i:
                corr_row.append(pose_correlations[j][i])
            elif j == i:
                corr_row.append(float(1))
            else:
                if mi is None:
                    corr_row.append(float(0))
                elif method == 'distance':
                    mj = get_pose_matrix(pj)
                    if mj is None:
                        corr_row.append(float(0))
                    else:
                        corr_row.append(mantel(mi, mj)[0])
                else: # method == 'laplacian'
                    mj = get_laplacian_matrix(pj, figure_index=0, figure_type='flipped_figures')
                    if mj is None:
                        corr_row.append(float(0))
                    else:
                        corr_row.append(1 - abs(np.subtract(mi.todense(), mj.todense()).sum()))
        pose_correlations.append(corr_row)

    return pose_correlations


def fill_nans_scipy1(padata, pkind='linear'):
    """ Fill in missing values from a time series, after the first non-NAN value
        and up to the last non-NAN value. Note that scipy.interpolated.interp1d
        provides a lot more options (splines, quadratic, etc.)
    """
    
    aindexes = np.arange(padata.shape[0])
    agood_indexes, = np.where(np.isfinite(padata))

    first_non_nan_index = agood_indexes[0] 
    last_non_nan_index = agood_indexes[-1]

    f = interp1d(agood_indexes
           , padata[agood_indexes]
           , bounds_error=False
           , copy=False
           , fill_value="extrapolate"
           , kind=pkind)

    interpolated = f(aindexes)
    interpolated[0:first_non_nan_index] = np.nan
    interpolated[last_non_nan_index+1:] = np.nan
    return interpolated


def movements_time_series(pose_data, pose_index=-1, method='distance', figure_type='flipped_figures', video_file=None):
    """ Calculate a time series of the differences between each pair of poses in a
        sequence. This works with a single figure (pose_index=0) or all the figures
        in the video (pose_index=-1). It can be run on its own, but typically this is
        a helper function for process_movement_series() (below).
    """
    
    per_frame_movements = []
    frame_timecodes = []

    pose_indices = []

    max_figures, total_time, total_figures = count_figures_and_time(pose_data, figure_type)

    threshold = .7

    #print("FIGURES PER FRAME IN TIME SERIES:",max_figures)

    # Typically the pose index is only specified if you know there's only one dancer
    # (in which case it's always 0)
    if pose_index != -1:
        max_figures = 1

    for f, frame in enumerate(pose_data):
        frame_movements = []
        frame_timecodes.append(frame['time'])
        if f < len(pose_data)-1:
            for p in range(max_figures):
                this_motion = np.nan
                movers = np.array([])
                # figure p must be available for both f and f-1
                # NOTE check p < len(pose_data[f][figure_type]) in case the data for that frame has been
                # truncated to [] (for example if extraneous data from the end of the video has been removed)
                if p < len(pose_data[f-1][figure_type]) and p < len(pose_data[f][figure_type]) and pose_data[f-1][figure_type][p].data.shape[0] != 0 and pose_data[f][figure_type][p].data.shape[0] != 0:
                    p1_conf = sum([c[2] for c in pose_data[f-1][figure_type][p].data]) / float(len(pose_data[f-1][figure_type][p].data))
                    p2_conf = sum([c[2] for c in pose_data[f][figure_type][p].data]) / float(len(pose_data[f][figure_type][p].data))
                    # XXX USE A BETTER CRITERION FOR SKIPPING POSES IF CONFIDENCE IS LOW
                    if p1_conf > threshold and p2_conf > threshold:
                        if method == 'distance':
                            plot_type = 'distance'
                            dmatrix1 = squareform(get_pose_matrix(pose_data[f-1], p, figure_type))
                            dmatrix2 = squareform(get_pose_matrix(pose_data[f], p, figure_type))
                            diffmatrix = np.absolute(dmatrix1 - dmatrix2)
                            movers = diffmatrix.sum(axis=1)
                            this_motion = movers.sum(axis=0) # For debugging
                        else:
                            plot_type = 'delaunay'
                            # Per-keypoint movements are not useful for Laplacian comparisons
                            similarity = compare_laplacians(pose_data[f-1], pose_data[f], p, figure_type)
                            # Can we get meaningful movement values if laplacians are of different sizes?
                            if similarity is not None:
                                movers = np.array([1 - similarity])
                
                frame_movements.append(movers)    
        
        per_frame_movements.append(frame_movements)

    return [per_frame_movements, frame_timecodes, max_figures]


def process_movement_series(pose_data, pose_index=-1, figure_type='flipped_figures', video_file=None, method='distance', interpolate=True, viz=True):
    """ Smooth, summarize, visualize movement data for one or more figures across a
        time series.
        Also visualize aggregate movement data for each keypoint, if distance matrix
        method is used.
    """
    
    print("GETTING MOVEMENT TIME SERIES")
    per_frame_results, frame_timecodes, max_figures = movements_time_series(pose_data, pose_index, method, figure_type, video_file)

    print("CALCULATING CHARACTERISTICS OF TIME SERIES")

    window_length = 5
    if video_file is not None:
        fps, total_frames = get_video_stats(video_file)
        window_length = max(window_length, int(round(fps/2.0)))

    movers = [] # To get the aggregate avg movement of each keypoint (not for Laplacian)
    movement_series = []
    frame_times = []

    smoothed_movement_series = []

    for j in range(max_figures):
        movement_series.append([])  
        smoothed_movement_series.append([])

    per_frame_movements = []
    
    for f, frame in enumerate(per_frame_results):
        frame_movements = np.zeros(TOTAL_COORDS)
        frame_times.append(frame_timecodes[f])
        for j in range(max_figures):
            if j >= len(frame) or frame[j].shape[0] == 0:
                #print("POSE",j,"HAS NO MOVEMENT DATA")
                movement_series[j].append(np.nan)
            elif method == 'distance':
                frame_movements = np.add(frame_movements, frame[j])
                movement_series[j].append(sum(frame[j]))
                movers.append(np.array(frame[j]))
            else: # method == 'laplacian'
                movement_series[j].append(frame[j][0])
            movers.append(frame[j])
        per_frame_movements.append(frame_movements)

    figure_time_series = np.array(movers)

    # Not sure if there's a meaningful way to aggregate the per-keypoint data
    # for the graph Laplacian approach (e.g., to be able to quantify how much
    # each keypoint moved during the video).
    if method == 'distance':
        movement_means = np.nanmean(figure_time_series, axis=0)
        movement_stdevs = np.nanstd(figure_time_series, axis=0)

    # Window length is half of fps (or ~5, whichever is larger)
    for j in range(max_figures):
        if interpolate:
            if np.isnan(np.nanmax(movement_series[j])):
                #print("Movement series is all nans, skipping")
                continue
            smoothed_movement_series[j] = smooth_series(fill_nans_scipy1(np.asarray(movement_series[j]), pkind='linear'),window_length).tolist()
        else:
            smoothed_movement_series[j] = smooth_series(np.array(movement_series[j]),window_length).tolist()

    if viz:
        print("VISUALIZING TIME SERIES CHARACTERISTICS")
    
        if method == 'distance':
            plt.figure()
            plt.xticks(np.arange(TOTAL_COORDS))

            plt.bar(np.arange(TOTAL_COORDS), movement_means)# yerr=movement_stdevs)

        fig = plt.figure(figsize=(12,6), constrained_layout=True)
        fig.dpi=100

        for j in range(len(smoothed_movement_series)):
            if (len(smoothed_movement_series[j]) > 0) and not np.isnan(np.nanmax(smoothed_movement_series[j])):
                plt.plot(frame_times,smoothed_movement_series[j]) #,color)
        plt.show()

    if method == 'distance':
        return [smoothed_movement_series, frame_times, per_frame_movements, movement_means, movement_stdevs]
    else:
        return [smoothed_movement_series, frame_times]